In [5]:
import awkward as ak
import uproot
import hist as hs

from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema, TreeMakerSchema, PHYSLITESchema

In [2]:
signaldomain     = 'root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root'
signalfile = uproot.open(signaldomain)

backgrounddomain = 'root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/csc/V1p171/Data2018_UL/v3/normalized/ParkingBPH4_Run2018A_goodLumi.root'
backgroundfile = uproot.open(backgrounddomain)

In [3]:
signalevents = NanoEventsFactory.from_root(
    signalfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

backgroundevents = NanoEventsFactory.from_root(
    backgroundfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

In [33]:
i = 0
for i in range(len(signalevents.fields)):
    if i<18:
        print(signalevents.fields[i], backgroundevents.fields[i])
    elif i>=18 and i<30:
        print(signalevents.fields[i],'NULL')
    else:
        try:
            print(signalevents.fields[i], backgroundevents.fields[i - 12])
        except:
            print(signalevents.fields[i])
    i+=1

runNum runNum
MC_condition MC_condition
lumiSec lumiSec
evtNum evtNum
rho rho
npv npv
Flag2_HBHENoiseFilter Flag2_HBHENoiseFilter
Flag2_HBHEIsoNoiseFilter Flag2_HBHEIsoNoiseFilter
Flag2_BadPFMuonFilter Flag2_BadPFMuonFilter
Flag2_globalSuperTightHalo2016Filter Flag2_globalSuperTightHalo2016Filter
Flag2_globalTightHalo2016Filter Flag2_globalTightHalo2016Filter
Flag2_BadChargedCandidateFilter Flag2_BadChargedCandidateFilter
Flag2_EcalDeadCellTriggerPrimitiveFilter Flag2_EcalDeadCellTriggerPrimitiveFilter
Flag2_ecalBadCalibFilter Flag2_ecalBadCalibFilter
Flag2_eeBadScFilter Flag2_eeBadScFilter
Flag2_all Flag2_all
metEENoise metEENoise
metPhiEENoise metPhiEENoise
gLLP_eta NULL
gLLP_phi NULL
gLLP_csc NULL
gLLP_dt NULL
gLLP_beta NULL
gLLP_e NULL
gLLP_pt NULL
gLLP_ctau NULL
gLLP_decay_vertex_r NULL
gLLP_decay_vertex_x NULL
gLLP_decay_vertex_y NULL
gLLP_decay_vertex_z NULL
nLeptons nLeptons
lepE lepE
lepPt lepPt
lepEta lepEta
lepPhi lepPhi
lepPdgId lepPdgId
lepDZ lepDZ
lepLooseId lepLooseId
le

In [17]:
for event in signalevents.cscRechitClusterPhi:
    if len(event) > 0:
        print(event)

[0.335]
[1.36]
[2.9]
[-1.13]
[1.95]
[-0.019]
[2.28]
[0.057]
[1.78]
[0.762]
[1.22]
[-2.47]
[-3.13]
[2.85]
[2.25]
[-1.16]
[0.647]
[2.18]
[-3.13]
[-0.259]
[-1.74]
[-0.906]
[-3.1]
[-0.276]
[2.71]
[-2.61]
[-2.27]
[2.61]
[-3.05]
[2.17]
[0.166]
[-0.0137, -2.89]
[-2.6]
[-0.279]
[3.11]
[-2.32]
[-1.15]
[2.51]
[-0.514]
[0.601]
[2.22]
[-1.41]
[1.65]
[-3]
[1.47]
[-2.73]
[2.15]
[-2.61]
[2.42]
[-0.345]
[1.65]
[-2.76]
[-0.297]
[-0.24]
[0.413, 2.56]
[-2.65]
[1.62]
[1.05]
[-0.162]
[2.23, 0.0752]
[-0.844]
[-2.83]
[-1.12]
[2.72]
[-0.0365]
[-0.948]
[1.76]
[-2.36]
[2.01]
[1.72]
[3.06]
[-0.887]
[-1.34]
[-1.24]
[-2.62]
[-2.48]
[2]
[2.69]
[3.11]
[-2.95]
[-0.193]
[0.715]
[2.36]
[0.739]
[3.1]
[-0.631]
[1.15]
[0.676]
[2.14]
[2.68]
[1.83]
[-1.67]
[0.51]
[-1.04]
[-1.79]
[1.21]
[0.605]
[1.76]
[-0.169]
[2.72]
[-2.5]
[-1.86]
[2.11]
[2.04]
[1.37]
[2.57]
[3.08]
[-2.2]
[1.81]
[-0.957]
[2.04]
[2.63]
[2.4]
[-2.15, -2.52]
[-1.45]
[-2.75]
[0.0546]
[-2.69]
[-0.93]
[0.0589]
[2.85]
[-0.751]
[1.87]
[2.19]
[-1]
[-0.433]
[-0.789]


In [26]:
[field for field in signalevents.fields if 'cscrechitcluster' in field.lower()]

['nCscRechitClusters',
 'cscRechitClusterX',
 'cscRechitClusterY',
 'cscRechitClusterZ',
 'cscRechitClusterTime',
 'cscRechitClusterTimeWeighted',
 'cscRechitClusterTimeTotal',
 'cscRechitClusterTimeSpread',
 'cscRechitClusterTimeSpreadWeighted',
 'cscRechitClusterTimeSpreadWeightedAll',
 'cscRechitClusterGenMuonDeltaR',
 'cscRechitClusterXYSpread',
 'cscRechitClusterMajorAxis',
 'cscRechitClusterMinorAxis',
 'cscRechitClusterEtaPhiSpread',
 'cscRechitClusterPhiSpread',
 'cscRechitClusterEtaSpread',
 'cscRechitClusterDeltaRSpread',
 'cscRechitClusterXSpread',
 'cscRechitClusterRSpread',
 'cscRechitClusterYSpread',
 'cscRechitClusterZSpread',
 'cscRechitClusterPhi',
 'cscRechitClusterEta',
 'cscRechitClusterSize',
 'cscRechitClusterMe11Ratio',
 'cscRechitClusterMe12Ratio',
 'cscRechitClusterNStation',
 'cscRechitClusterNStation5',
 'cscRechitClusterNStation10',
 'cscRechitClusterNStation10perc',
 'cscRechitClusterAvgStation',
 'cscRechitClusterAvgStation5',
 'cscRechitClusterAvgStation1

In [25]:
signalevents.metEENoise

<Array [44.2, 16.9, 28.7, ... 17.1, 56.9, 50.1] type='823789 * float32[parameter...'>

In [13]:
temp = hs.Hist.new.Reg(30, -200, 200, name='clsTime', label='time').Double()
temp.fill(ak.flatten(backgroundevents.cscRechitClusterTimeWeighted, axis=None)) 


Hist(Regular(30, -200, 200, name='clsTime', label='time'), storage=Double()) # Sum: 7515384.0 (7515447.0 with flow)